In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.simplefilter('ignore')

In [2]:
fs_df = pd.read_excel('/Users/seanyu_mac/Downloads/주문내역_51273_2024-05-08~2025-05-08.xlsx')
na_df = pd.read_csv('/Users/seanyu_mac/Downloads/네이버스마트스토어 공구 - 시트1 (3).csv')

In [3]:
fs_df = fs_df[fs_df['상품 번호'] == 51273]
fs_df['구분'] = '친한스토어'
fs_df['결제 완료일'] = fs_df['결제 완료일'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [4]:
na_df['price'] = 0
con2 = na_df['옵션정보'] == '선택: 2박스'
con4 = na_df['옵션정보'] == '선택: 4박스'
con8 = na_df['옵션정보'] == '선택: 8박스'
na_df.loc[con2, 'price'] = 69000
na_df.loc[con4, 'price'] = 125000
na_df.loc[con8, 'price'] = 222000

In [5]:
na_df['매출'] = na_df['수량'] * na_df['price']
na_df['구분'] = '네이버스토어'
na_df['주문일시'] = pd.to_datetime(na_df['주문일시'])
na_df['주문일시'] = na_df['주문일시'].apply(lambda x: x.strftime('%Y-%m-%d'))

na_df['박스수량'] = na_df['옵션정보'].apply(lambda x: x.split('박스')[0].split(': ')[1].strip())
na_df['박스수량'] = na_df['박스수량'].astype(int)

In [6]:
fs_df.columns

Index(['결제 번호', '상품 번호', '주문 번호', '결제 완료일', '주문 상태', '상품명', '옵션명', '수량', '정상가',
       '판매가', '쿠폰 할인', '포인트 할인', '적립금', '정산가', '수령인명', '수령인 연락처', '구매자명',
       '구매자 ID', '구매자 연락처', '주소', '우편번호', '배송 메모', '주문 확인일', '발주 요청일',
       '배송 완료일', '구매 확정일', '배송택배사', '송장 번호', '배송비 정책', '배송비', '도서산간 배송비',
       '취소 완료일', '판매자 취소일', '반품 요청일', '수거 요청일', '수거 완료일', '반품 완료일', '구분'],
      dtype='object')

In [7]:
fs_df['박스수량'] = fs_df['옵션명'].apply(lambda x: x.split('박스')[0])
fs_df['박스수량'] = fs_df['박스수량'].astype(int)
fs_df = fs_df[['결제 번호', '주문 번호', '결제 완료일', '주문 상태', '상품명', '옵션명', '수량', '박스수량', '정산가', '구분']]
fs_df.rename(columns={'정산가': '매출', '수량': '주문수량'}, inplace=True)

In [8]:
na_df = na_df[['상품주문번호', '주문번호', '주문일시', '주문상태', '상품명', '옵션정보', '수량', '박스수량', '매출', '구분']]

In [9]:
na_df.columns = fs_df.columns

In [10]:
df = pd.concat([fs_df, na_df])

In [11]:
df['주문 상태'].unique()

array(['결제완료', '배송중'], dtype=object)

In [12]:
status = ['결제완료', '배송중']

In [13]:
cancel_df = df[~df['주문 상태'].isin(status)]
df = df[df['주문 상태'].isin(status)]

In [14]:
cancel_df

,결제 번호,주문 번호,결제 완료일,주문 상태,상품명,옵션명,주문수량,박스수량,매출,구분


In [15]:
df.loc[df['옵션명'].str.contains('2'), '옵션명'] = '2박스'
df.loc[df['옵션명'].str.contains('4'), '옵션명'] = '4박스'
df.loc[df['옵션명'].str.contains('8'), '옵션명'] = '8박스'
df

,결제 번호,주문 번호,결제 완료일,주문 상태,상품명,옵션명,주문수량,박스수량,매출,구분
0,parma20250508100234mNu17,55023-53646,2025-05-08,결제완료,[공동구매] 샤인약사 x YDY 콜라겐 비오틴 듀오 / 2박스 4박스 8박스,2박스,1,2,69000,친한스토어
0,2025050959984561,2025050968987501,2025-05-09,결제완료,[공동구매] 샤인약사 x YDY 콜라겐 비오틴 듀오 / 2박스 4박스 8박스,4박스,1,4,125000,네이버스토어
1,2025050958232731,2025050967858251,2025-05-09,결제완료,[공동구매] 샤인약사 x YDY 콜라겐 비오틴 듀오 / 2박스 4박스 8박스,4박스,1,4,125000,네이버스토어
2,2025050952528201,2025050964116481,2025-05-09,결제완료,[공동구매] 샤인약사 x YDY 콜라겐 비오틴 듀오 / 2박스 4박스 8박스,4박스,1,4,125000,네이버스토어
3,2025050952001211,2025050963755551,2025-05-09,결제완료,[공동구매] 샤인약사 x YDY 콜라겐 비오틴 듀오 / 2박스 4박스 8박스,4박스,1,4,125000,네이버스토어
4,2025050949509371,2025050962192651,2025-05-09,결제완료,[공동구매] 샤인약사 x YDY 콜라겐 비오틴 듀오 / 2박스 4박스 8박스,8박스,1,8,222000,네이버스토어
5,2025050948667251,2025050961658851,2025-05-09,결제완료,[공동구매] 샤인약사 x YDY 콜라겐 비오틴 듀오 / 2박스 4박스 8박스,2박스,1,2,69000,네이버스토어
6,2025050847693631,2025050861038781,2025-05-08,결제완료,[공동구매] 샤인약사 x YDY 콜라겐 비오틴 듀오 / 2박스 4박스 8박스,8박스,1,8,222000,네이버스토어
7,2025050847512251,2025050860921051,2025-05-08,결제완료,[공동구매] 샤인약사 x YDY 콜라겐 비오틴 듀오 / 2박스 4박스 8박스,2박스,1,2,69000,네이버스토어
8,2025050845402541,2025050859543551,2025-05-08,결제완료,[공동구매] 샤인약사 x YDY 콜라겐 비오틴 듀오 / 2박스 4박스 8박스,4박스,1,4,125000,네이버스토어


In [16]:
pivot_df = pd.DataFrame(pd.pivot_table(df, index=['결제 완료일', '옵션명'], values=['매출', '주문수량', '박스수량'], aggfunc='sum', margins=True, margins_name='합계'))
pivot_df = pivot_df.applymap(lambda x: f'{x:,}')
pivot_df

매출 박스수량 주문수량
결제 완료일     옵션명                     
2025-05-08 2박스    276,000    8    4
           4박스  1,125,000   36    9
           8박스  2,220,000   80   10
2025-05-09 2박스     69,000    2    1
           4박스    500,000   16    4
           8박스    222,000    8    1
합계              4,412,000  150   29